In [301]:
from openai import OpenAI
from prompt_toolkit.key_binding.bindings.named_commands import downcase_word

from secret import OPENAI_API_KEY
from PIL import Image
import base64
import os
import json
import re
import pandas as pd
import io
from tabulate import tabulate

In [302]:
client = OpenAI(api_key=OPENAI_API_KEY)
model = "gpt-4o-mini"

In [303]:
def encode_image(image_path, max_size=(512, 512), quality=80):
    image = Image.open(image_path)

    # Redimensionner l'image
    image.thumbnail(max_size)

    # Convertir en bytes avec compression
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG", quality=quality)

    # Encoder en Base64
    encoded_string = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return encoded_string

In [304]:
def send_chat_request(message):
    try:
        response = client.chat.completions.create(
            model=model, messages=message
        )

        result = response.choices[0].message.content.strip()
        result = extract_json(result)

        tokens = response.usage.total_tokens

        return result, tokens


    except Exception as e:
        print(f"Erreur OpenAI : {e}")
        return -1, None

In [305]:
def chat_get_key_words(image_paths):

    # Liste pour chaque image et chaque texte associé
    content_list = []
    for image_path in image_paths:
        base64_image = encode_image(image_path)
        image_name = os.path.basename(image_path)
        content_list.append({
            "type": "text",
            "text": f"""Décris moi l'image avec 5 mots-clés.Retourne le résultat au format JSON : {{ {image_name} : [mot-clé1, mot-clé2, mot-clé3, mot-clé4, mot-clé5] }}
            Si tu ne trouves pas de mot-clé, renvoie {{ {image_name} : "nan" }}"""
        })
        content_list.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
        })

    messages = [
        {
            "role": "user",
            "content": content_list
        }
    ]

    return send_chat_request(messages)


In [306]:
def extract_json(response_text):
    """
    Extrait la portion JSON (délimitée par {}) de la réponse textuelle pour seulement avoir le dictionnaire et non le texte généré par l'ia.
    """
    match = re.search(r'\{.*\}', response_text, re.DOTALL)
    if match:
        json_str = match.group()
        try:
            return json.loads(json_str)
        except Exception as e:
            print(f"Erreur lors du chargement du JSON : {e}")
            return None
    else:
        print("Aucun JSON trouvé dans la réponse.")
        return None

In [307]:
def chat_get_categories(keywords_output):
    """
    Utilise les mots-clés extraits pour regrouper les images similaires en catégories.
    Les images sont identifiées par leur ordre dans la liste.
    """

    # Préparation d'un prompt détaillé incluant le résultat des mots-clés et l'ordre des images
    prompt = f"""Voici les listes de mots-clés obtenues pour chaque image (dans l'ordre) : {keywords_output}
    Regroupe les images similaires dans des catégories. Une catégorie est décrite par un seul mot-clé. Une image ne peut appartenir qu'à une seule catégorie. Retourne le résultat au format JSON : {{ "categorie1": [ "name", "name" ], "categorie2": [ "name", "name" ],...}}"""

    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]

    return send_chat_request(messages)

In [308]:
directory = "test_data"
allowed_extensions = {".jpg", ".jpeg", ".png"}
image_paths = [
    os.path.join(directory, filename)
    for filename in os.listdir(directory)
    if os.path.splitext(filename)[1].lower() in allowed_extensions
]

## Ajout des données au DataFrame

In [309]:
def create_df(image_paths):
    image_list = []
    for path in image_paths:
        image = Image.open(path)
        image_name = os.path.basename(path)
        exifdata = image._getexif()
        date_time, localisation = None, None
        if exifdata:
            for tag_id, value in exifdata.items():
                tag = Image.ExifTags.TAGS.get(tag_id, tag_id)
                if tag == "DateTime":
                    date_time = value
                elif tag == "GPSInfo":
                    localisation = value

            image_list.append((image_name, path, date_time, localisation))

        else:
            print("Aucune donnée EXIF trouvée.")

    df = pd.DataFrame(image_list, columns=["image_name", "path", "date_time", "localisation"])
    df["keywords"] = ""
    df["categories"] = ""

    return df

In [310]:
def add_keywords_to_df(image_data, keywords_output):
    if keywords_output:
        # Mise à jour uniquement pour les images présentes dans keywords_output
        image_data["keywords"] = image_data.apply(
            lambda row: keywords_output[row["image_name"]]
            if row["image_name"] in keywords_output else row["keywords"], axis=1
        )
    else:
        print("Aucun mot clé fourni ! ")
    return image_data


In [311]:
def add_categories_to_df(image_data, categories_output):
    if categories_output:
        #Inversion du dict : on associe une categorie a chaque image
        image_to_categories = {img: cat for cat, images in categories_output.items() for img in images}

        image_data.loc[image_data["image_name"].isin(image_to_categories.keys()), "categories"] = image_data["image_name"].map(image_to_categories)
    else:
        print("Aucune catégorisation trouvée !")

    return image_data

In [312]:
def get_missing_values(dictionnary):
    missing_values = {}
    for key, value in dictionnary.items():
        if value is None or value == "" or value == "nan" or value == "None":
            missing_values.update({key: value})

    return missing_values

In [313]:
def get_missing_path(paths, dictionnary):
    missing_paths = []
    for path in paths:
        image_name = os.path.split(path)[-1]
        #print(f"image_name : {image_name}")
        if image_name in dictionnary.keys() or path in dictionnary.keys():
            missing_paths.append(path)

    return missing_paths

In [314]:
def get_missing_values_path(paths, dictionnary):
    missing_values = get_missing_values(dictionnary)
    missing_paths = get_missing_path(paths, missing_values)
    print(f"Images détectées avec valeurs manquantes : {missing_values.keys()}")
    print(f"Chemins renvoyés pour traitement : {missing_paths}")
    return missing_paths

In [315]:
def pipeline_keywords(image_paths, limit_size=10):
    image_data = create_df(image_paths)
    new_image_paths = image_paths[:]
    total_keywords_tokens = 0

    all_keywords = False
    keywords_output = {}
    only_once = True

    while not all_keywords :
        #print(f"Entree dans la boucle avec {new_image_paths}")
        for i in range(0, len(new_image_paths), limit_size):
            interval = [i, min(i + limit_size, len(image_paths))]
            subset_image_paths = new_image_paths[interval[0]:interval[1]]
            print(f"Image paths : {subset_image_paths}")

            keywords_output, keywords_tokens = chat_get_key_words(subset_image_paths)
            print(f"Keywords : {keywords_output}")
            total_keywords_tokens += keywords_tokens

            image_data = add_keywords_to_df(image_data, keywords_output)

            print(f"Total tokens : {total_keywords_tokens}")

        if only_once:
            fake_image_path = "photos_victor/IMG_20241228_132157.jpg"
            keywords_output[fake_image_path] = "nan"
            new_image_paths.append(fake_image_path)

            new_row = pd.DataFrame({"image_name" : "IMG_20241228_132157.jpg","path": [fake_image_path]})
            image_data = pd.concat([image_data, new_row], ignore_index=True)

            only_once = False

        new_image_paths = get_missing_values_path(new_image_paths, keywords_output)
        print(f"Images à traiter après filtrage : {new_image_paths}")


        if not new_image_paths :
            all_keywords = True

    return image_data, total_keywords_tokens

In [316]:
image_data, keywords_tokens = pipeline_keywords(image_paths, 10)

Image paths : ['test_data\\20240822_142034.jpg', 'test_data\\20240822_142036.jpg', 'test_data\\20240822_142048.jpg', 'test_data\\20240825_121144.jpg', 'test_data\\20240826_184159.jpg', 'test_data\\20240827_084810.jpg', 'test_data\\20240827_084914.jpg', 'test_data\\20240827_092136.jpg', 'test_data\\20240828_141857.jpg', 'test_data\\20240828_174310.jpg']
Keywords : {'20240822_142034.jpg': ['bâtiment', 'brouillard', 'urbain', 'routes', 'arbres'], '20240822_142036.jpg': ['bâtiment', 'moderne', 'fenêtres', 'architecture', 'urbain'], '20240822_142048.jpg': ['bâtiment', 'historique', 'pierres', 'voitures', 'rond-point'], '20240825_121144.jpg': ['beurre', 'cacahuète', 'pot', 'animal', 'épicerie'], '20240826_184159.jpg': ['sentier', 'forêt', 'nature', 'plantes', 'jouer'], '20240827_084810.jpg': ['bâtiment', 'herbe', 'architecture', 'extérieur', 'verre'], '20240827_084914.jpg': 'nan', '20240827_092136.jpg': ['évenement', 'tente', 'bacs', 'groupe', 'activités'], '20240828_141857.jpg': ['bâtiment'

In [317]:
print(tabulate(image_data, headers="keys", tablefmt="psql"))

+----+-------------------------+---------------------------------------+---------------------+---------------------------------------------------------------------+---------------------------------------------------------------------+--------------+
|    | image_name              | path                                  | date_time           | localisation                                                        | keywords                                                            | categories   |
|----+-------------------------+---------------------------------------+---------------------+---------------------------------------------------------------------+---------------------------------------------------------------------+--------------|
|  0 | 20240822_142034.jpg     | test_data\20240822_142034.jpg         | 2024:08:22 14:20:34 |                                                                     | ['bâtiment', 'brouillard', 'urbain', 'routes', 'arbres']            |              |


In [229]:
copy_image_data = image_data.copy()

In [230]:
def pipeline_categories(image_data, limit_size=200):
    keywords = image_data.set_index("image_name")["keywords"].to_dict()
    total_categories_tokens = 0

    for i in range(0, len(keywords), limit_size):
        interval = [i, min(i + limit_size, len(image_data))]
        subset_keys = list(keywords.keys())[interval[0]:interval[1]]

        subset_keywords = {key: keywords[key] for key in subset_keys}

        categories_output, categories_tokens = chat_get_categories(subset_keywords)
        total_categories_tokens += categories_tokens

        image_data = add_categories_to_df(image_data, categories_output)

    return image_data, total_categories_tokens

In [231]:
copy_image_data, categories_tokens = pipeline_categories(copy_image_data)

In [232]:
print(tabulate(copy_image_data, headers="keys", tablefmt="psql"))

+----+-------------------------+---------------------------------------+---------------------+---------------------------------------------------------------------+--------------------------------------------------------------------------------+----------------+
|    | image_name              | path                                  | date_time           | localisation                                                        | keywords                                                                       | categories     |
|----+-------------------------+---------------------------------------+---------------------+---------------------------------------------------------------------+--------------------------------------------------------------------------------+----------------|
|  0 | 20240822_142034.jpg     | test_data\20240822_142034.jpg         | 2024:08:22 14:20:34 |                                                                     | ['bâtiments', 'rue', 'arbre', 'ciel nuageux', 